In [1]:
from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pickle
import os
import re
from sklearn.metrics import roc_curve, auc
import numpy as np
from gensim.models import word2vec
import logging

In [3]:
dros_strict_real = pickle.load(open('Results/drosophila/strict_real.pkl', 'rb'))
dros_gen_real = pickle.load(open('Results/drosophila/gen_real.pkl', 'rb'))
dros_be_real = pickle.load(open('Results/drosophila/be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [8]:
for seed in random_seeds:
    real_tr_te_name = 'dros_real_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(dros_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)
    tr_val_name = 'dros_tr_val_split_' + str(seed)
    real_train_data, b, c, d = pred.manual_train_test_split(train_data, tr_val_name, random_state=seed, test_set_prop=0.2)

In [11]:
dimensions = [100, 200, 300, 400, 500, 600]
for i in dimensions:
    w2v_parameters = [i, 5, 4, 6, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'dros_strict_dim_parameter_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'dros_gen_dim_parameter_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'dros_be_dim_parameter_' + str(i), w2v_parameters)

2017-05-06 21:34:39,562 : INFO : collecting all words and their counts
2017-05-06 21:34:39,563 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-06 21:34:39,599 : INFO : collected 13177 word types from a corpus of 185681 raw words and 7271 sentences
2017-05-06 21:34:39,600 : INFO : Loading a fresh vocabulary
2017-05-06 21:34:39,614 : INFO : min_count=5 retains 3537 unique words (26% of original 13177, drops 9640)
2017-05-06 21:34:39,615 : INFO : min_count=5 leaves 170112 word corpus (91% of original 185681, drops 15569)
2017-05-06 21:34:39,624 : INFO : deleting the raw counts dictionary of 13177 items
2017-05-06 21:34:39,626 : INFO : sample=0.001 downsamples 36 most-common words
2017-05-06 21:34:39,626 : INFO : downsampling leaves estimated 123095 word corpus (72.4% of prior 170112)
2017-05-06 21:34:39,627 : INFO : estimated required memory for 3537 words and 100 dimensions: 4598100 bytes
2017-05-06 21:34:39,638 : INFO : resetting layer weights
2017-05-

Parsing datasets sentences
Training Word2Vec Model


2017-05-06 21:34:40,096 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-06 21:34:40,098 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-06 21:34:40,103 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-06 21:34:40,106 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-06 21:34:40,107 : INFO : training on 928405 raw words (615514 effective words) took 0.4s, 1441981 effective words/s
2017-05-06 21:34:40,107 : INFO : precomputing L2-norms of word weight vectors
2017-05-06 21:34:40,131 : INFO : saving Word2Vec object under Results/dros_strict_dim_parameter_100_model, separately None
2017-05-06 21:34:40,132 : INFO : not storing attribute syn0norm
2017-05-06 21:34:40,132 : INFO : not storing attribute cum_table
2017-05-06 21:34:40,166 : INFO : saved Results/dros_strict_dim_parameter_100_model


Parsing datasets sentences


2017-05-06 21:34:40,549 : INFO : collecting all words and their counts
2017-05-06 21:34:40,550 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-06 21:34:40,592 : INFO : PROGRESS: at sentence #10000, processed 244981 words, keeping 13594 word types
2017-05-06 21:34:40,635 : INFO : PROGRESS: at sentence #20000, processed 488356 words, keeping 18196 word types
2017-05-06 21:34:40,677 : INFO : PROGRESS: at sentence #30000, processed 731064 words, keeping 21332 word types
2017-05-06 21:34:40,709 : INFO : collected 23335 word types from a corpus of 901524 raw words and 37017 sentences
2017-05-06 21:34:40,710 : INFO : Loading a fresh vocabulary
2017-05-06 21:34:40,737 : INFO : min_count=5 retains 8246 unique words (35% of original 23335, drops 15089)
2017-05-06 21:34:40,738 : INFO : min_count=5 leaves 876113 word corpus (97% of original 901524, drops 25411)


...

2017-05-06 21:37:38,366 : INFO : collected 23335 word types from a corpus of 901524 raw words and 37017 sentences
2017-05-06 21:37:38,367 : INFO : Loading a fresh vocabulary
2017-05-06 21:37:38,396 : INFO : min_count=5 retains 8246 unique words (35% of original 23335, drops 15089)
2017-05-06 21:37:38,397 : INFO : min_count=5 leaves 876113 word corpus (97% of original 901524, drops 25411)
2017-05-06 21:37:38,422 : INFO : deleting the raw counts dictionary of 23335 items
2017-05-06 21:37:38,424 : INFO : sample=0.001 downsamples 39 most-common words
2017-05-06 21:37:38,425 : INFO : downsampling leaves estimated 646591 word corpus (73.8% of prior 876113)
2017-05-06 21:37:38,425 : INFO : estimated required memory for 8246 words and 600 dimensions: 43703800 bytes
2017-05-06 21:37:38,455 : INFO : resetting layer weights
2017-05-06 21:37:38,622 : INFO : training model with 4 workers on 8246 vocabulary and 600 features, using sg=0 hs=0 sample=0.001 negative=5 window=6
2017-05-06 21:37:38,623 : 

Parsing datasets sentences


2017-05-06 21:37:48,510 : INFO : collecting all words and their counts
2017-05-06 21:37:48,511 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-06 21:37:48,558 : INFO : PROGRESS: at sentence #10000, processed 231388 words, keeping 14641 word types
2017-05-06 21:37:48,606 : INFO : PROGRESS: at sentence #20000, processed 463186 words, keeping 20111 word types
2017-05-06 21:37:48,653 : INFO : PROGRESS: at sentence #30000, processed 694962 words, keeping 23963 word types
2017-05-06 21:37:48,702 : INFO : PROGRESS: at sentence #40000, processed 925654 words, keeping 27029 word types


Training Word2Vec Model


2017-05-06 21:37:48,749 : INFO : PROGRESS: at sentence #50000, processed 1157257 words, keeping 29704 word types
2017-05-06 21:37:48,799 : INFO : PROGRESS: at sentence #60000, processed 1387247 words, keeping 32225 word types
2017-05-06 21:37:48,848 : INFO : PROGRESS: at sentence #70000, processed 1618178 words, keeping 34360 word types
2017-05-06 21:37:48,893 : INFO : PROGRESS: at sentence #80000, processed 1848004 words, keeping 36287 word types
2017-05-06 21:37:48,950 : INFO : PROGRESS: at sentence #90000, processed 2077948 words, keeping 38042 word types
2017-05-06 21:37:49,003 : INFO : PROGRESS: at sentence #100000, processed 2309347 words, keeping 39767 word types
2017-05-06 21:37:49,057 : INFO : PROGRESS: at sentence #110000, processed 2541111 words, keeping 41317 word types
2017-05-06 21:37:49,110 : INFO : PROGRESS: at sentence #120000, processed 2772131 words, keeping 42710 word types
2017-05-06 21:37:49,169 : INFO : PROGRESS: at sentence #130000, processed 3002172 words, keep

In [4]:
for i in dimensions:
    w2w_model_strict = word2vec.Word2Vec.load('Results/dros_strict_dim_parameter_'+str(i)+'_model')
    w2w_model_gen = word2vec.Word2Vec.load('Results/dros_gen_dim_parameter_'+str(i)+'_model')
    w2w_model_be = word2vec.Word2Vec.load('Results/dros_be_dim_parameter_'+str(i)+'_model')
    
    for seed in random_seeds:
        data_name = 'Results/dros_tr_val_split_' + str(seed)
        train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
        train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
        validation_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
        validation_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_strict, feature_count=i)
        
        strict_list_SR_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_gen, feature_count=i)
        
        strict_list_GEN_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                      train_labels, validation_labels]
        
        w2v_train_vecs, w2v_val_vecs = pred.word_2_vec_feat_vecs(train_data, validation_data, w2w_model_be, feature_count=i)
        
        strict_list_BE_dims_param = [w2v_train_vecs, w2v_val_vecs,
                                     train_labels, validation_labels]
        
        pickle.dump(strict_list_SR_dims_param, open('Results/dros_strict_list_SR_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_GEN_dims_param, open('Results/dros_strict_list_GEN_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        pickle.dump(strict_list_BE_dims_param, open('Results/dros_strict_list_BE_dims_param_'+str(i)+'_'+str(seed)+'_results_list.pkl', 'wb'))
        
        strict_final_list = [strict_list_SR_dims_param, 
                             strict_list_GEN_dims_param, 
                             strict_list_BE_dims_param]
        print ('\nPredicting\n')
        errors = []
        fpr = []
        tpr = []

        for entry in strict_final_list:
            error_w2v_norm, probs_w2v_norm = pred.XGB_classifier(entry[0], entry[1],
                                                                 entry[2], entry[3])
            fpr_w2v_norm, tpr_w2v_norm, _ = roc_curve(entry[3], probs_w2v_norm)
            error_w2v_fs, probs_w2v_fs = pred.XGB_classifier(entry[0], entry[1],
                                                             entry[2], entry[3],
                                                             feature_selection=True)
            fpr_w2v_fs, tpr_w2v_fs, _ = roc_curve(entry[3], probs_w2v_fs)

            errors.append([error_w2v_norm, error_w2v_fs])
            fpr.append([fpr_w2v_norm, fpr_w2v_fs])
            tpr.append([tpr_w2v_norm, tpr_w2v_fs])
            
        pickle.dump(errors, open('Results/dros_dim_param'+str(i)+'_errors_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(fpr, open('Results/dros_dim_param'+str(i)+'_fpr_pickle_'+str(seed)+'.pkl', 'wb'))
        pickle.dump(tpr, open('Results/dros_dim_param'+str(i)+'_tpr_pickle_'+str(seed)+'.pkl', 'wb'))

/home/bioinformatics/Desktop/KU/Master_Thesis/Code/Proper_Class/2.0/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


/home/bioinformatics/Desktop/KU/Master_Thesis/Code/Proper_Class/2.0/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


/home/bioinformatics/Desktop/KU/Master_Thesis/Code/Proper_Class/2.0/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)


...


Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


/home/bioinformatics/Desktop/KU/Master_Thesis/Code/Proper_Class/2.0/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


/home/bioinformatics/Desktop/KU/Master_Thesis/Code/Proper_Class/2.0/prediction.py:174: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting


 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions

 Fitting XGBoost Model!

 Making Predictions


In [12]:
def mult_open(direct, pattern):
    pickle_list = []
    file_list = []
    for f in os.listdir(direct):
        if re.search(pattern, f):
            f = f.split('_')
            f[-1] = f[-1][:-4]
            file_list.append(f)
    file_list.sort(key = lambda x: int(x[-1]))
    for file in file_list:
        file = '_'.join(file)
        file = file + '.pkl'
        pkl = pickle.load(open(os.path.join(direct, file), 'rb'))
        pickle_list.append(pkl)
    return pickle_list

In [13]:
for i in dimensions:
    drct = 'Results/W2V_Parameter_Search/Dims/'
    errors_dros = mult_open(drct, str(i)+'_errors_')
    fpr_dros = mult_open(drct, str(i)+'_fpr_')
    tpr_dros = mult_open(drct, str(i)+'_tpr_')
    
    mean_err_dros_strict = []
    mean_auc_dros_strict = []
    mean_err_dros_gen = []
    mean_auc_dros_gen = []
    mean_err_dros_be = []
    mean_auc_dros_be = []
    for e, f, t in zip(errors_dros, fpr_dros, tpr_dros):
        input_list = [[e, f, t]]
        name_list = ['drosophila']
        for idx in range(3):
            for item, name in zip(input_list, name_list):
                for fpr_item, tpr_item, error_item in zip(item[1][idx], item[2][idx], item[0][idx]):
                    roc_auc = auc(fpr_item, tpr_item)
                    auc_val = '%.3f' % roc_auc
                    error = '%.3f' % error_item
                    if idx == 0:
                        mean_err_dros_strict.append(error_item)
                        mean_auc_dros_strict.append(roc_auc)
#                         legend_label = 'Strict error ' + name + ' auc=' + str(auc_val) + ' error=' + str(error)
#                         print(legend_label)
                    elif idx == 1:
                        mean_err_dros_gen.append(error_item)
                        mean_auc_dros_gen.append(roc_auc)
#                         legend_label = 'Gen error ' + name + ' auc=' + str(auc_val) + ' error=' + str(error)
#                         print(legend_label)
                    elif idx == 2:
                        mean_err_dros_be.append(error_item)
                        mean_auc_dros_be.append(roc_auc)
#                         legend_label = 'BE error ' + name + ' auc=' + str(auc_val) + ' error=' + str(error)
#                         print(legend_label)
#                 print('\n')

    mean_err_dros_org_strict = mean_err_dros_strict[0::2]
    mean_err_dros_fs_strict = mean_err_dros_strict[1::2]
    mean_auc_dros_org_strict = mean_auc_dros_strict[0::2]
    mean_auc_dros_fs_strict = mean_auc_dros_strict[1::2]

    mean_err_dros_org_gen = mean_err_dros_gen[0::2]
    mean_err_dros_fs_gen = mean_err_dros_gen[1::2]
    mean_auc_dros_org_gen = mean_auc_dros_gen[0::2]
    mean_auc_dros_fs_gen = mean_auc_dros_gen[1::2]

    mean_err_dros_org_be = mean_err_dros_be[0::2]
    mean_err_dros_fs_be = mean_err_dros_be[1::2]
    mean_auc_dros_org_be = mean_auc_dros_be[0::2]
    mean_auc_dros_fs_be = mean_auc_dros_be[1::2]
    
    print('Dims = '+str(i)+' Drosophila mean strict error original - ', np.mean(mean_err_dros_org_strict))
    print('Dims = '+str(i)+' Drosophila mean strict error feature selection - ', np.mean(mean_err_dros_fs_strict))
    print('Dims = '+str(i)+' Drosophila mean strict AUC original - ', np.mean(mean_auc_dros_org_strict))
    print('Dims = '+str(i)+' Drosophila mean strict AUC feature selection - ', np.mean(mean_auc_dros_fs_strict))
    print('\n')
    print('Dims = '+str(i)+' Drosophila mean gen error original - ', np.mean(mean_err_dros_org_gen))
    print('Dims = '+str(i)+' Drosophila mean gen error feature selection - ', np.mean(mean_err_dros_fs_gen))
    print('Dims = '+str(i)+' Drosophila mean gen AUC original - ', np.mean(mean_auc_dros_org_gen))
    print('Dims = '+str(i)+' Drosophila mean gen AUC feature selection - ', np.mean(mean_auc_dros_fs_gen))
    print('\n')
    print('Dims = '+str(i)+' Drosophila mean BE error original - ', np.mean(mean_err_dros_org_be))
    print('Dims = '+str(i)+' Drosophila mean BE error feature selection - ', np.mean(mean_err_dros_fs_be))
    print('Dims = '+str(i)+' Drosophila mean BE AUC original - ', np.mean(mean_auc_dros_org_be))
    print('Dims = '+str(i)+' Drosophila mean BE AUC feature selection - ', np.mean(mean_auc_dros_fs_be))
    print('\n')

Dims = 100 Drosophila mean strict error original -  0.351453359042
Dims = 100 Drosophila mean strict error feature selection -  0.349877380232
Dims = 100 Drosophila mean strict AUC original -  0.48823523874
Dims = 100 Drosophila mean strict AUC feature selection -  0.493827666301


Dims = 100 Drosophila mean gen error original -  0.352627205145
Dims = 100 Drosophila mean gen error feature selection -  0.352746845854
Dims = 100 Drosophila mean gen AUC original -  0.498002964969
Dims = 100 Drosophila mean gen AUC feature selection -  0.504285094557


Dims = 100 Drosophila mean BE error original -  0.34692404945
Dims = 100 Drosophila mean BE error feature selection -  0.345171406626
Dims = 100 Drosophila mean BE AUC original -  0.495656337464
Dims = 100 Drosophila mean BE AUC feature selection -  0.492639575514


Dims = 200 Drosophila mean strict error original -  0.34775566577
Dims = 200 Drosophila mean strict error feature selection -  0.347167134976
Dims = 200 Drosophila mean strict AU

In [6]:
org_100 = 0.48823523874
fs_100 = 0.493827666301

org_200 = 0.518242149813
fs_200 = 0.519622695748

org_300 = 0.49458909605
fs_300 = 0.495140712247

org_400 = 0.530355182715
fs_400 = 0.522848523164

org_500 = 0.51306193792
fs_500 = 0.498997252443

org_600 = 0.531561847862
fs_600 = 0.51873939355

res_list = [org_100, fs_100, org_200, fs_200, org_300, fs_300, org_400, fs_400, org_500, fs_500, org_600, fs_600]
 
print(res_list.index(max(res_list))) #STRICT - ORG 600

10


In [7]:
org_100 = 0.498002964969
fs_100 = 0.504285094557

org_200 = 0.506520367903
fs_200 = 0.520416510331

org_300 = 0.497951234897
fs_300 = 0.488438900647

org_400 = 0.504174192718
fs_400 = 0.501883436351

org_500 = 0.495890341956
fs_500 = 0.496831193417

org_600 = 0.507628510361
fs_600 = 0.503696115449

res_list = [org_100, fs_100, org_200, fs_200, org_300, fs_300, org_400, fs_400, org_500, fs_500, org_600, fs_600]
 
print(res_list.index(max(res_list))) #GEN - FS 200

3


In [8]:
org_100 = 0.495656337464
fs_100 = 0.492639575514

org_200 = 0.49458909605
fs_200 = 0.485498559817

org_300 = 0.498330663255
fs_300 = 0.483447994672

org_400 = 0.491053228877
fs_400 = 0.495965522422

org_500 = 0.498901271933
fs_500 = 0.49154261637

org_600 = 0.486860508996
fs_600 = 0.48583941366

res_list = [org_100, fs_100, org_200, fs_200, org_300, fs_300, org_400, fs_400, org_500, fs_500, org_600, fs_600]
 
print(res_list.index(max(res_list))) #BE - ORG 500

8


In [ ]:
min_word_count = [1, 2, 3, 4, 5, 6]
for i in min_word_count:
    w2v_parameters = [500, i, 4, 6, 0.001]
    strict = pred.make_w2v_model(dros_strict_real, 'dros_strict_min_word_count_' + str(i), w2v_parameters)
    gen = pred.make_w2v_model(dros_gen_real, 'dros_gen_min_word_count_' + str(i), w2v_parameters)
    be = pred.make_w2v_model(dros_be_real, 'dros_be_min_word_count_' + str(i), w2v_parameters)

In [13]:
data_name = 'Results/dros_tr_val_split_' + str(144)
train_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))

In [14]:
len(train_data)

212

In [15]:
len(dros_strict_real)

7271